In [1]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('/home/amelia/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
import geopandas as gp
 
# from urbansim_templates import modelmanager as mm
# from urbansim_templates.models import LargeMultinomialLogitStep
# import os; os.chdir('/home/juan/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
# from matplotlib import pyplot as plt
# import matplotlib.animation as animation
# import seaborn as sns

# from scripts import datasources, models, variables, utils
# scripts = '~/ual_model_workspace/spring-2019-models/scripts'

In [2]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [3]:
bay_counties = [1, 13, 97, 95, 81, 85, 75, 41, 55]

In [4]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 96.17 s
Total time to execute iteration 1 with iteration value None: 96.17 s


In [5]:
os.listdir('/home/data/CHTS_csv_format/data')

['Deliv_HH.csv',
 'Deliv_ACTIVITY.csv',
 'Deliv_LD.csv',
 'LookUp_LD.csv',
 'LookUp_Home.csv',
 'ASSN_TravelDate.csv',
 'LookUp_PLACE.csv',
 'LookUp_PER.csv',
 'Deliv_PER.csv',
 'Deliv_VEH.csv',
 'Deliv_PLACE.csv']

In [6]:
place = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PLACE.csv')
person = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PER.csv')
activity = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_ACTIVITY.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv').rename({"parcel_id": "school_parcel_id"}, axis=1)
students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
parcels = orca.get_table('parcels').to_frame()
tracts = gp.read_file('/home/amelia/ual_model_workspace/spring-2019-models/notebooks-amelia/files/San_Francisco_Bay_Region_2010_Census_Tracts.shp')
TAZs = gp.read_file('/home/amelia/ual_model_workspace/spring-2019-models/notebooks-amelia/files/Transportation_Analysis_Zones.shp')





In [7]:
TAZs["centroid"] = TAZs["geometry"].apply(lambda x: x.centroid)
TAZs["Longitude"] = TAZs["centroid"].apply(lambda x: x.x)
TAZs["Latitude"] = TAZs["centroid"].apply(lambda x: x.y)
tracts["centroid"] = tracts["geometry"].apply(lambda x: x.centroid)
tracts["Longitude"] = tracts["centroid"].apply(lambda x: x.x)
tracts["Latitude"] = tracts["centroid"].apply(lambda x: x.y)


1588

#### 1.  Filter for just the k-12 students in the CHTS persons table. grade field is not available in table, so use age <= 19 as a proxy



In [9]:
chts_stu = person.loc[(person['STUDE'].isin([1, 2])) & (person["AGE"] <= 19)]
print(len(person))
print(len(chts_stu))

chts_stu.head()

109113
20838


,SAMPN,PERNO,RELAT,GEND,AGE,AGEB,HISP,RACE1,RACE2,RACE3,...,SCTFIP,STRACT,WPrimaryCity,WSTFIP,W2PrimaryCity,W2STFIP,SPrimaryCity,SSTFIP,PERWGT,EXPPERWGT
4,1032036,3,3,1,15,NaN,2,1.0,97.0,NaN,...,73.0,17030.0,NaN,NaN,NaN,NaN,SAN DIEGO,6.0,0.941412,318.965100
5,1032036,4,3,1,14,NaN,2,1.0,97.0,NaN,...,73.0,17032.0,NaN,NaN,NaN,NaN,SAN DIEGO,6.0,0.941412,318.965100
6,1032036,5,3,1,7,NaN,2,1.0,97.0,NaN,...,73.0,17030.0,NaN,NaN,NaN,NaN,SAN DIEGO,6.0,0.941412,318.965100
9,1032053,3,3,2,9,NaN,1,97.0,NaN,NaN,...,47.0,1602.0,NaN,NaN,NaN,NaN,MERCED,6.0,0.059297,20.090794
10,1032053,4,3,2,7,NaN,1,97.0,NaN,NaN,...,47.0,1602.0,NaN,NaN,NaN,NaN,MERCED,6.0,0.059297,20.090794


#### 2. In Place table: create unique record identifiers by person, place and hh

In [35]:
place['HHPER'] = place['SAMPN'].map(str) + place['PERNO'].map(str) #sampn is hh identifier, perno is the person num within
#the household

place['HHPERPLA'] = place['HHPER'].map(str) + place['PLANO'].map(str)
place1 = place[['SAMPN', 'HHPER','HHPERPLA','MODE',"ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN","TRIPDUR", "CITY", 'PNAME', 
                'STATE', 'ZIP', 'XCORD', 'YCORD',"TRACT"]]
place1.head()

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR,CITY,PNAME,STATE,ZIP,XCORD,YCORD,TRACT
0,1031985,10319851,103198511,NaN,3,0,8,40,NaN,VALLEJO,HOME,CA,94591,HIDDEN,HIDDEN,252202.0
1,1031985,10319851,103198512,6.0,9,2,12,53,22.0,BENICIA,BENICIA SENIOR CITIZEN CENTER,CA,94510,HIDDEN,HIDDEN,960500.0
2,1031985,10319851,103198513,6.0,13,13,2,59,20.0,VALLEJO,HOME,CA,94591,HIDDEN,HIDDEN,252202.0
3,1031985,10319852,103198521,NaN,3,0,9,16,NaN,VALLEJO,HOME,CA,94591,HIDDEN,HIDDEN,252202.0
4,1031985,10319852,103198522,5.0,9,26,10,43,10.0,VALLEJO,LOMA VISTA ELEMENTARY SCHOOL,CA,94589,HIDDEN,HIDDEN,950200.0


In [ ]:
place.columns[-100:]
place[['SAMPN', 'PERNO', 'PLANO', PNAME', 'CITY', 'STATE', 'ZIP', 'XCORD', 'YCORD', 'RouteDistance',
       'CTFIP', 'TRACT', 'PPrimaryCity', 'PSTFIP', 'PERWGT', 'EXPPERWGT',
       'TRIPNO', 'TCF', 'TCFPERWGT', 'EXPTCFPERWGT', 'HHPER', 'HHPERPLA']]

#### 3. In activity table: create unique record identifiers by person, place and hh id

In [15]:
activity['HHPER'] = activity['SAMPN'].map(str) + activity['PERNO'].map(str)
activity['HHPERPLA'] = activity['HHPER'].map(str) + activity['PLANO'].map(str)
activity['count'] = activity.groupby('HHPERPLA').cumcount() +1 #group by unique person-hh-place key, to account 
#for the same person going to the same place but doing different activities, and therefore having different
#trip purposes. cumcount() +1 numbers these different activities for the same person-place trip
activity2 = activity.pivot(index='HHPERPLA', columns='count', values='APURP').reset_index()
# #then take these person-place grouped entries and articulate the different activity purposes (up to 3)
activity2.columns = ['HHPERPLA', 'APURP1', 'APURP2', 'APURP3']
activity2["HHPER"] = activity2["HHPERPLA"].apply(lambda x: x[:-1])
activity3=pd.merge(activity2, place1[["HHPERPLA", "PNAME", "TRACT", "ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN",
                                      "TRIPDUR"]], on='HHPERPLA')

activity3.head()


,HHPERPLA,APURP1,APURP2,APURP3,HHPER,PNAME,TRACT,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR
0,103198511,1.0,2.0,1.0,10319851,HOME,252202.0,3,0,8,40,NaN
1,103198512,37.0,NaN,NaN,10319851,BENICIA SENIOR CITIZEN CENTER,960500.0,9,2,12,53,22.0
2,103198513,2.0,7.0,8.0,10319851,HOME,252202.0,13,13,2,59,20.0
3,103198521,1.0,2.0,7.0,10319852,HOME,252202.0,3,0,9,16,NaN
4,103198522,33.0,NaN,NaN,10319852,LOMA VISTA ELEMENTARY SCHOOL,950200.0,9,26,10,43,10.0


#### 4. Classify trip purpose as a school trip or not 
if at least one of the 3 trip purposes is for school/studying (5), school/classroom/laboratory (17), meals at school/college (18), and also After-school or non-class-related
sports/physical activities (19),  All other after-school or non-class related activities (library, music rehearsal, clubs, etc.) (20). 

In [36]:
#place column is "school" if at least one of the 3 trip purpose columns is 5 (school/study trip purpose)
school_purposes = [5.0, 17.0, 18.0, 19.0, 20.0, 5, 17, 18, 19, 20] #what about 22 drop off passengers?
work_purposes = [9.0, 10.0, 16.0, 9, 10, 16]
 

def assign_trip_type(row):
    return assign_trip_type_helper(row["APURP1"], row["APURP2"], row["APURP3"], row["PNAME"])

def assign_trip_type_helper(apurp1, apurp2, apurp3, pname):
    if ((apurp1 in school_purposes) | (apurp2 in school_purposes) |
        (apurp3 in school_purposes)):
        return "school"

    elif ((apurp1 in work_purposes) | (apurp2 in work_purposes) | 
          (apurp3 in work_purposes)):
        return "work"
    elif (pname == 'HOME'):
        return "home"
    else:
        return "other"
    
def assign_origin_trip_type(row):
    if (row.HHPER == row.prev_HHPER):
        return assign_trip_type_helper(row.prev_APURP1, row.prev_APURP2, row.prev_APURP3, row.prev_PNAME)
    else:
        return "no origin trip"
        

Filter out trips for people that are never home


In [93]:
# def home_trips(df): 

#     num_home =  len(df[df["trip_type"] == 'home'])

#     return num_home > 0

# ishome = activity3.groupby("HHPER").apply(home_trips)
# home_hhper = ishome[ishome] #select the people for whome ishome is true
# activity4 = activity3[activity3["HHPER"].isin(home_hhper.index)]
# activity4.head()

,HHPERPLA,APURP1,APURP2,APURP3,HHPER,PNAME,TRACT,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR,trip_type
0,103198511,1.0,2.0,1.0,10319851,HOME,252202.0,3,0,8,40,NaN,home
1,103198512,37.0,NaN,NaN,10319851,BENICIA SENIOR CITIZEN CENTER,960500.0,9,2,12,53,22.0,other
2,103198513,2.0,7.0,8.0,10319851,HOME,252202.0,13,13,2,59,20.0,home
3,103198521,1.0,2.0,7.0,10319852,HOME,252202.0,3,0,9,16,NaN,home
4,103198522,33.0,NaN,NaN,10319852,LOMA VISTA ELEMENTARY SCHOOL,950200.0,9,26,10,43,10.0,other


Filter out non school trips 


In [96]:
activity3['trip_type'] = activity3.apply(assign_trip_type, axis=1)
school_activity = activity3[activity3["trip_type"] == "school"]
print(len(school_activity))
print(len(activity3))
school_activity.head()

17479
460520


,HHPERPLA,APURP1,APURP2,APURP3,HHPER,PNAME,TRACT,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR,trip_type
68,103205332,17.0,NaN,NaN,10320533,REYES ELEMENTARY SCHOOL,1602.0,7,33,16,10,3.0,school
74,103205342,17.0,NaN,NaN,10320534,REYES ELEMENTARY SCHOOL,1602.0,7,33,16,10,3.0,school
80,103205352,17.0,NaN,NaN,10320535,REYES ELEMENTARY SCHOOL,1602.0,7,33,16,10,3.0,school
148,103519825,17.0,NaN,NaN,10351982,"CALIFORNIA STATE UNIVERSITY, SAN BERNARDINO",4510.0,16,50,19,15,20.0,school
183,103536442,19.0,NaN,NaN,10353644,JOHN F KENNEDY HIGH SCHOOL,4005.0,16,40,17,46,4.0,school


These functions that create field for the previous trip are helpful if you want to filter by the trip origin or previous trip type. For simplicity this model is assuming all K-12 school trips are from home

In [37]:
# activity3["diff_person"] = activity3.HHPER != activity3.HHPER.shift(-1)
# activity3["prev_HHPER"] = activity3.HHPER.shift(1)
# activity3["prev_APURP1"] = activity3.APURP1.shift(1)
# activity3["prev_APURP2"] = activity3.APURP2.shift(1)
# activity3["prev_APURP3"] = activity3.APURP3.shift(1)
# activity3["prev_PNAME"] = activity3.PNAME.shift(1)
# activity3["prev_TRACT"] = activity3.TRACT.shift(1)
# activity3["origin_trip_type"] = activity3.apply(assign_origin_trip_type, axis=1) #this isn't quite right, assign origin trip
#row needs to be looking at the record above



#### 5. Merge students dataframe with activity dataframe filtered by school trips only, on the unique person id (hhper). 
Persons dataframe includes several demographic variables for students, including race(s), nativity, relationship in family, gender, age, ethnicity, and others that are not currently included but could be.

In [143]:
students['HHPER'] = students['SAMPN'].map(str) + students['PERNO'].map(str)
students.columns[:100]
place2 =  pd.merge(students, school_activity, on='HHPER', how='right')[['SAMPN', 'PERNO', 'RELAT', 'GEND', 'AGE', 'HISP', 'RACE1', 'RACE2',
       'RACE3', 'RACE4', 'O_RACE', 'NTVTY', 'STUDE', 'SCITY_persons', 'SSTAT',
       'SZIP_persons', 'SMODE', 'EDUCA', 'SNAME_lookup', 'SCITY_lookup',
       'SZIP_lookup', 'SCTFIP_lookup', 'STRACT_lookup', 'SBLOCK',
       'SPrimaryCity_lookup', 'SSTFIP_lookup', 'HCITY', 'HZIP', 'HXCORD',
       'HYCORD', 'HBLOCK', 'HTRACT', 'HCTFIP', 'HPrimaryCity',
       'parcel_id_home', 'parcel_id_work', 'zone_id_home', 'zone_id_work',
       'school_id', 'HHPER', 'HHPERPLA', 'APURP1', 'APURP2', 'APURP3', 'PNAME',
       'trip_type', "ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN", "TRIPDUR"]]
place2 = place2.rename({"HHPER_x": "HHPER", "PNAME_x": "PNAME"}, axis=1)
place2.head()

,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,...,APURP1,APURP2,APURP3,PNAME,trip_type,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR
0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,17.0,NaN,NaN,ST CHARLES CHURCH,school,7,50,15,0,20.0
1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,1.0,5.0,2.0,HOME,school,3,0,8,1,NaN
2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,17.0,18.0,NaN,PRESIDIO HILL SCHOOL,school,8,15,15,20,14.0
3,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,5.0,2.0,8.0,HOME,school,15,31,2,59,11.0
4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,NaN,...,17.0,20.0,NaN,NOVATO HIGH SCHOOL,school,7,15,18,45,15.0


#### 6. Assign locations to schools: merge with the schools table on the school id

In [153]:
def get_zone_id(parcel_id):
    '''gets the zone_id (TAZ) of school and home locations by the parcel id'''
    try: 
        zone_id = parcels.loc[parcel_id]["zone_id"]

    except Exception as e:
        zone_id = np.nan
#         print(e)
        
    return zone_id

In [154]:
schools_rel = schools.drop(['sw_rank_2108',
       'sw_rank_2017', 'sw_rank_2016', 'ss_rank_2018_', 'ss_rank_2017',
       'ss_rank_2016'], axis=1)
stu_schools_trips = pd.merge(place2, schools_rel, on="school_id", how='left')
stu_schools_trips["school_zone_id"] = stu_schools_trips["school_parcel_id"].apply(get_zone_id)

stu_schools_trips.head()




,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,...,Ungraded Elementary Enrollmnet,Grade 9 Enrollment,Grade 10 Enrollment,Grade 11 Enrollment,Grade 12 Enrollment,Ungraded Secondary Enrollment,Total Enrollment,nodeID,school_parcel_id,school_zone_id
0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,279.0,5.413314e+08,1160691.0,309.0
1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,220.0,2.589679e+08,1019806.0,99.0
2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,220.0,2.589679e+08,1019806.0,99.0
3,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,220.0,2.589679e+08,1019806.0,99.0
4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,NaN,...,0.0,401.0,360.0,321.0,296.0,0.0,1378.0,2.807231e+09,809605.0,1409.0


#### Change format of arrival and departure times to easily calculate dwell time at school. Filter for only morning trips (defined as before 11am)

In [155]:
stu_schools_trips['activity_start'] = stu_schools_trips['ARR_HR'] + (stu_schools_trips['ARR_MIN']/60)
stu_schools_trips['activity_end'] = stu_schools_trips['DEP_HR'] + (stu_schools_trips['DEP_MIN']/60)
morning_school_trips = stu_schools_trips[stu_schools_trips["activity_start"] < 11]
morning_school_trips['calc_trip_dwell'] = stu_schools_trips['activity_end'] - stu_schools_trips['activity_start']
morning_school_trips.head()


,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,...,Grade 11 Enrollment,Grade 12 Enrollment,Ungraded Secondary Enrollment,Total Enrollment,nodeID,school_parcel_id,school_zone_id,activity_start,activity_end,calc_trip_dwell
0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,279.0,5.413314e+08,1160691.0,309.0,7.833333,15.000000,7.166667
1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,220.0,2.589679e+08,1019806.0,99.0,3.000000,8.016667,5.016667
2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,220.0,2.589679e+08,1019806.0,99.0,8.250000,15.333333,7.083333
4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,NaN,...,321.0,296.0,0.0,1378.0,2.807231e+09,809605.0,1409.0,7.250000,18.750000,11.500000
6,1173970.0,3.0,3.0,1.0,8.0,1.0,1.0,NaN,NaN,NaN,...,0.0,0.0,0.0,626.0,3.455715e+08,812949.0,1407.0,8.333333,15.533333,7.200000


In [ ]:
morning_school_trips1 = morning_school_trips.rename({"SMODE_1.0": "walk", "SMODE_2.0": "bike", 
                                  "SMODE_4.0": "other_non_motor", "SMODE_5.0": "auto_driver", 
                "SMODE_6.0": "auto_passenger", "SMODE_7.0": "car_van_pool", "SMODE_10.0": "rental_car", 
      "SMODE_14.0": "o_priv_transit", "SMODE_15.0": "local_rapid_bus", "SMODE_16.0": "exp_comm_bus",
    
     "SMODE_18.0": "school_bus", "SMODE_19.0": "pub_shuttle", "SMODE_23.0": "o_bus", "SMODE_24.0": "bart_metro_r_pu",
             "SMODE_26.0": "metro_bl_gr_go", "SMODE_27.0": "street_cable_car", "SMODE_99.0": "RF"}, axis=1)
morning_school_trips_modes1.head()

#### Get Mode dummy column

In [156]:
morning_school_trips_modes = pd.get_dummies(morning_school_trips, columns=["SMODE"]) #not sure if this column or mode column
morning_school_trips_modes1 = morning_school_trips_modes.rename({"SMODE_1.0": "walk", "SMODE_2.0": "bike", 
                                  "SMODE_4.0": "other_non_motor", "SMODE_5.0": "auto_driver", 
                "SMODE_6.0": "auto_passenger", "SMODE_7.0": "car_van_pool", "SMODE_10.0": "rental_car", 
      "SMODE_14.0": "o_priv_transit", "SMODE_15.0": "local_rapid_bus", "SMODE_16.0": "exp_comm_bus",
    
     "SMODE_18.0": "school_bus", "SMODE_19.0": "pub_shuttle", "SMODE_23.0": "o_bus", "SMODE_24.0": "bart_metro_r_pu",
             "SMODE_26.0": "metro_bl_gr_go", "SMODE_27.0": "street_cable_car", "SMODE_99.0": "RF"}, axis=1)
morning_school_trips_modes1.head()


,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,...,o_priv_transit,local_rapid_bus,exp_comm_bus,school_bus,pub_shuttle,o_bus,bart_metro_r_pu,metro_bl_gr_go,street_cable_car,RF
0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6,1173970.0,3.0,3.0,1.0,8.0,1.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [157]:
print(len(morning_school_trips.columns))
print(len(morning_school_trips_modes.columns))
print(len(morning_school_trips_modes.columns))

97
113
113


In [158]:
morning_school_trips_modes1.columns[80:]

Index(['Grade 6 Enrollment', 'Grade 7 Enrollment', 'Grade 8 Enrollment',
       'Ungraded Elementary Enrollmnet', 'Grade 9 Enrollment',
       'Grade 10 Enrollment', 'Grade 11 Enrollment', 'Grade 12 Enrollment',
       'Ungraded Secondary Enrollment', 'Total Enrollment', 'nodeID',
       'school_parcel_id', 'school_zone_id', 'activity_start', 'activity_end',
       'calc_trip_dwell', 'walk', 'bike', 'other_non_motor', 'auto_driver',
       'auto_passenger', 'car_van_pool', 'rental_car', 'o_priv_transit',
       'local_rapid_bus', 'exp_comm_bus', 'school_bus', 'pub_shuttle', 'o_bus',
       'bart_metro_r_pu', 'metro_bl_gr_go', 'street_cable_car', 'RF'],
      dtype='object')

Check Mode Frequencies

In [159]:
just_modes = morning_school_trips_modes1[['walk', 'bike', 'other_non_motor', 'auto_driver',
       'auto_passenger', 'car_van_pool', 'rental_car', 'o_priv_transit',
       'local_rapid_bus', 'exp_comm_bus', 'school_bus', 'pub_shuttle', 'o_bus',
       'bart_metro_r_pu', 'metro_bl_gr_go', 'street_cable_car', 'RF']]
# just_modes.sum(axis=0).sort_values(ascending=False)



In [150]:
# for c in morning_school_trips_modes1.columns:
#     print(c)
len(morning_school_trips_modes1.columns)

112

In [160]:
morning_school_trips_modes1.to_csv("school_trips_modes_clean.csv")
morning_school_trips.to_csv("school_trips_clean.csv")

#### Below not currently used stuff

#### Functions to approximate the TAZ of locations from the census tract information (not currently used, instead using location info from schools table)

In [ ]:
#get school location from school_id
from sklearn.neighbors import BallTree

def parcel_id(data):
    ''' Assigns a parcel ID given a x and y coordinate
    Input: DataFrame with x and y coordinates
    Output: Array-like Parcel_ID'''
    #Transforming Latitude/Longitude to radians
    parcels_rad = np.deg2rad(parcels[['y', 'x']])
    data_rad = np.deg2rad(data)
    
    tree = BallTree(parcels_rad, metric='haversine')
    
    dists, idxs = tree.query(data_rad, return_distance=True)
    
    return parcels.iloc[idxs[:,0]].index

def get_zone_id(parcel_id):
    '''gets the zone_id (TAZ) of school and home locations by the parcel id'''
    try: 
        zone_id = parcels.loc[parcel_id]["zone_id"]

    except Exception as e:
        zone_id = np.nan
#         print(e)
        
    return zone_id

In [ ]:
display(stu_schools_trips.head())
print("non unique hhper ids of students matched with schools:", len(stu_schools_trips))
print("unique hhper ids of students matched with schools: ", len(stu_schools_trips["HHPER"].unique()))

#### Approximate the trip origin locations by matching the centroid of the census tract with its block id, and relating that to a TAZ. Place Name, school name, block id, and lat lon information have been redacted from CHTS so this is only an approximation of the location with the information available

In [ ]:
tracts["TRACT"] = tracts["trctid"].apply(lambda x: float(x[4:])/100)
school_home_trips_tract = pd.merge(tracts, school_home_trips, right_on='STRACT', left_on='TRACT')[[
       'SAMPN', 'HHPER', 'HHPERPLA', 'MODE', 'ARR_HR', 'ARR_MIN', 'DEP_HR',
       'DEP_MIN', 'PNAME', 'APURP1', 'APURP2', 'APURP3', 'trip_type',
       'trip_origin', 'STUDE','SCITY', 'STRACT',
       'SZIP', 'SMODE', 'fipco', 
        'trctid', "Latitude", "Longitude",'TRACT']]
print(school_home_trips_tract.columns[-100:])
school_home_trips_tract.head()

In [ ]:
# school_home_trips_tract["place_parcel_id*"] = parcel_id(school_home_trips_tract[["Latitude", "Longitude"]])
# school_home_trips_tract["place_taz*"] = school_home_trips_tract["place_parcel_id*"].apply(lambda x: get_zone_id(x))

# school_home_trips_tract.head(10)


In [ ]:
# #eliminate records for persons that are never home

# neverhome = trips[trips.home_count.isnull()]
# neverathome = neverhome["HHPER"].unique()
# trips1 = trips[~trips.HHPER.isin(neverathome)] 


# # create a trip id for each trip that is 2 numbers where the first is the number of the school trip and the sceond
# # is the number of the home trip

# trips1["trip_ID"] = trips1["school_count"].map(str) + trips1["home_count"].map(str)
# trips1["HHPERTRIP"] = trips1["HHPER"].map(str) + (trips1["trip_ID"]).map(str)
# trips1["HHPERTRIP"] = trips1["HHPERTRIP"].map(float)/100
# trips1['num_trips'] = trips1.groupby('HHPERTRIP').cumcount() + 1 #adds a column with the cumulative count of trips taken
# #by that person that day. 12 is the max.
# trips1.head()


In [ ]:
#isolate just the trips that are the start of home-school or school-home trip (place column is the start of the trip)
# home_school_trips = trips1.loc[trips['place'].isin(["home","school"])]
# school_trips = trips1.loc[trips['place'].isin(["school"])]

# school_trips.head()

In [ ]:
# home_school_trips['mode_from'] = home_school_trips.groupby('HHPER', group_keys=False).MODE.shift(-1)
# home_school_trips.rename({"SMODE": "to_mode"}, inplace=True, axis=1)
# home_school_trips.head()
